In [1]:
from hstrader import HsTrader
from hstrader.models import OrderType, SideType, Resolution

In [2]:
CLIENT_ID = "<YOUR_CLIENT_ID>"
CLIENT_SECRET = "<YOUR_CLIENT_SECRET>"
URL = "<YOUR_BROKER_URL>"

In [3]:
client = HsTrader(CLIENT_ID, CLIENT_SECRET, URL)

In [4]:
symbol = client.get_symbol("EURUSD")

In [29]:
data = client.get_market_history(symbol.id,count_back=1000,resolution=Resolution.H1)

In [30]:
import pandas as pd

In [31]:
df = pd.DataFrame()

In [32]:
bars = [bar.model_dump() for bar in data ]

In [33]:
df = pd.DataFrame(bars)
df['time'] = pd.to_datetime(df['time'], unit='s')

df

,time,open,high,low,close,volume
0,2024-08-04 19:59:00+00:00,1.09097,1.09240,1.09097,1.09229,6323.550178
1,2024-08-04 20:00:00+00:00,1.09229,1.09240,1.09107,1.09240,3888.037294
2,2024-08-04 21:00:00+00:00,1.09240,1.09240,1.09163,1.09201,122687.873880
3,2024-08-04 22:00:00+00:00,1.09201,1.09201,1.09022,1.09060,228773.319674
4,2024-08-04 23:00:00+00:00,1.09060,1.09113,1.09036,1.09100,263188.104116
...,...,...,...,...,...,...
668,2024-09-13 17:00:00+00:00,1.10835,1.10967,1.10800,1.10815,216373.801095
669,2024-09-13 18:00:00+00:00,1.10815,1.10967,1.10745,1.10779,231731.024121
670,2024-09-13 19:00:00+00:00,1.10779,1.10813,1.10739,1.10781,230932.360395
671,2024-09-13 20:00:00+00:00,1.10781,1.10967,1.10724,1.10732,181396.835047


In [34]:
# Calculate the 20-period Simple Moving Average (SMA)
df['SMA'] = df['close'].rolling(window=20).mean()

In [35]:
# Calculate the 20-period Standard Deviation (SD)
df['SD'] = df['close'].rolling(window=20).std()

In [36]:
# Calculate the Upper Bollinger Band (UB) and Lower Bollinger Band (LB)
df['UB'] = df['SMA'] + 2 * df['SD']
df['LB'] = df['SMA'] - 2 * df['SD']

In [37]:
import plotly.graph_objs as go

In [38]:
fig = go.Figure()

# Add the price chart
fig.add_trace(go.Scatter(x=df.index, y=df['close'], mode='lines', name='Price'))

# Add the Upper Bollinger Band (UB) and shade the area
fig.add_trace(go.Scatter(x=df.index, y=df['UB'], mode='lines', name='Upper Bollinger Band', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df.index, y=df['LB'], fill='tonexty', mode='lines', name='Lower Bollinger Band', line=dict(color='green')))

# Add the Middle Bollinger Band (MA)
fig.add_trace(go.Scatter(x=df.index, y=df['SMA'], mode='lines', name='Middle Bollinger Band', line=dict(color='blue')))

# Customize the chart layout
fig.update_layout(title='AAPL Stock Price with Bollinger Bands',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  showlegend=True)
